
Final Report Generator for AA_VA

This script generates a comprehensive final PDF report that includes:
- Executive Summary
- Test Overview and Statistics
- LLM Event Log with screenshots and OCR
- Activity Map and Details
- Feature Analysis (Unique vs Generic)
- Exploration Statistics
- OCR Images Section

#!/usr/bin/env python3

In [ ]:

import os
import sys
import json
import time
import uuid
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict


# Add the project root to the Python path

In [ ]:
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))

from loguru import logger

# Assuming these modules exist in your project structure
# from src.vision.engine import VisionEngine
# from src.ai.openai_client import OpenAIClient

# Mock imports if actual modules are not available for isolated testing

In [ ]:
class VisionEngine:
    def __init__(self):
        pass
    def process_screenshot(self, screenshot_path: str):
        return {"elements": []}

class OpenAIClient:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super(OpenAIClient, cls).__new__(cls)
        return cls._instance

    def __init__(self):
        if not hasattr(self, '_initialized'):
            self._initialized = True
            pass # Initialize your OpenAI client here if needed
    
    @classmethod
    def instance(cls):
        if not cls._instance:
            cls._instance = cls()
        return cls._instance


# PDF Generation imports

Represents a single LLM event during app exploration.

Represents a unique app state.

Represents an Android activity with its states.

Generates comprehensive final PDF reports for AA_VA.

Initialize the report generator.
        
        Parameters
        ----------
        output_dir : str
            Directory to save the final report
        app_name : str
            Name of the app being tested
        package_name : str
            Package name of the app being tested

In [ ]:
try:
    from reportlab.lib.pagesizes import letter, A4
    from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak, KeepTogether
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.lib.units import inch
    from reportlab.lib import colors
    from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT, TA_JUSTIFY
    from reportlab.pdfgen import canvas
    from reportlab.lib.utils import ImageReader
    REPORTLAB_AVAILABLE = True
except ImportError:
    REPORTLAB_AVAILABLE = False
    logger.warning("ReportLab not available. Install with: pip install reportlab")


class LLMEvent:
    
    def __init__(self, timestamp: str, state_id: str, element_id: str, 
                 action: str, input_text: str = "", scroll_direction: str = "",
                 reasoning: str = "", resulting_state_id: str = "",
                 screenshot_path: str = "", ocr_image_path: str = "",
                 llm_response_image_path: str = "", state_transition_before: str = "",
                 state_transition_after: str = "", visual_description: str = ""):
        self.timestamp = timestamp
        self.state_id = state_id
        self.element_id = element_id
        self.action = action
        self.input_text = input_text
        self.scroll_direction = scroll_direction
        self.reasoning = reasoning
        self.resulting_state_id = resulting_state_id
        self.screenshot_path = screenshot_path
        self.ocr_image_path = ocr_image_path
        self.llm_response_image_path = llm_response_image_path
        self.state_transition_before = state_transition_before
        self.state_transition_after = state_transition_after
        self.visual_description = visual_description


class AppState:
    
    def __init__(self, state_id: str, activity_name: str, description: str = "",
                 screenshot_path: str = "", elements: List[Dict] = None,
                 visual_description: str = ""):
        self.state_id = state_id
        self.activity_name = activity_name
        self.description = description
        self.screenshot_path = screenshot_path
        self.elements = elements or []
        self.entry_timestamp = datetime.now().isoformat()
        self.visual_description = visual_description


class Activity:
    
    def __init__(self, name: str, package_name: str = ""):
        self.name = name
        self.package_name = package_name
        self.states: List[AppState] = []
        self.primary_function = ""
        self.entry_state: Optional[AppState] = None


class FinalReportGenerator:
    
    def __init__(self, output_dir: str = "final_reports", app_name: str = "Unknown App", package_name: str = "unknown.package"):

        # Auto-generate output directory based on package name

In [ ]:
        if package_name != "unknown.package":

            # Convert package name to directory format: com.example.app -> test_com_example_app

In [ ]:
            dir_name = f"test_{package_name.replace('.', '_')}"
            self.output_dir = Path(output_dir) / dir_name
        else:
            self.output_dir = Path(output_dir)
        
        self.output_dir.mkdir(parents=True, exist_ok=True)
        

        # Store app information

In [ ]:
        self.package_name = package_name
        

        # Auto-extract app name from package if not provided or is generic

In [ ]:
        if app_name in ["Unknown App", "Activity", "Main"] or app_name == "Unknown App":
            self.app_name = self._extract_app_name_from_package(package_name)
        else:
            self.app_name = app_name
        

        # Create subdirectories

In [ ]:
        (self.output_dir / "images").mkdir(exist_ok=True)
        (self.output_dir / "graphs").mkdir(exist_ok=True)
        (self.output_dir / "ocr_images").mkdir(exist_ok=True)
        

        # Initialize components

In [ ]:
        self.vision_engine = VisionEngine()
        self.ai_client = OpenAIClient.instance()
        

        # Data structures

In [ ]:
        self.llm_events: List[LLMEvent] = []
        self.app_states: Dict[str, AppState] = {}
        self.activities: Dict[str, Activity] = {}
        self.transition_graph = nx.DiGraph()
        

        # Test duration tracking

In [ ]:
        self.test_start_time: Optional[datetime] = None
        self.test_end_time: Optional[datetime] = None
        self.exploration_start_time = datetime.now()  # Report generation start time
        

        # Statistics

Set the actual test start time.

Set the actual test end time.

Get the formatted test duration.

Add an LLM event to the log.

In [ ]:
        self.total_actions = 0
        self.unique_elements_interacted = set()
        
        logger.info(f"Initialized Final Report Generator")
        logger.info(f"Output directory: {self.output_dir}")
        logger.info(f"App: {self.app_name} ({self.package_name})")
    
    def set_test_start_time(self, start_time: datetime) -> None:
        self.test_start_time = start_time
        logger.info(f"Test start time set to: {start_time}")
    
    def set_test_end_time(self, end_time: datetime) -> None:
        self.test_end_time = end_time
        logger.info(f"Test end time set to: {end_time}")
    
    def get_test_duration(self) -> str:
        if self.test_start_time and self.test_end_time:
            duration = self.test_end_time - self.test_start_time
            total_seconds = int(duration.total_seconds())
            hours = total_seconds // 3600
            minutes = (total_seconds % 3600) // 60
            seconds = total_seconds % 60
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        elif self.test_start_time: # If test ended recently, calculate from start to now
            duration = datetime.now() - self.test_start_time
            total_seconds = int(duration.total_seconds())
            hours = total_seconds // 3600
            minutes = (total_seconds % 3600) // 60
            seconds = total_seconds % 60
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        else: # Fallback to exploration time if test times are not set
            exploration_end_time = datetime.now()
            duration = exploration_end_time - self.exploration_start_time
            total_seconds = int(duration.total_seconds())
            hours = total_seconds // 3600
            minutes = (total_seconds % 3600) // 60
            seconds = total_seconds % 60
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    
    def add_llm_event(self, event: LLMEvent) -> None:
        self.llm_events.append(event)
        self.total_actions += 1
        self.unique_elements_interacted.add(event.element_id)
        

        # Add to transition graph

Add an app state.

In [ ]:
        self.transition_graph.add_edge(
            event.state_id, 
            event.resulting_state_id,
            action=event.action,
            element=event.element_id,
            input_text=event.input_text
        )
    
    def add_app_state(self, state: AppState) -> None:
        self.app_states[state.state_id] = state
        

        # Add to activity

In [ ]:
        if state.activity_name not in self.activities:
            self.activities[state.activity_name] = Activity(state.activity_name)
        
        activity = self.activities[state.activity_name]
        activity.states.append(state)
        

        # Set as entry state if it's the first state in this activity

Format timestamp to be more readable.

Get appropriate icon for action type.

Simplify element ID to be more readable.

Create more descriptive reasoning based on action and element type.

In [ ]:
        if not activity.entry_state:
            activity.entry_state = state
    
    def _format_timestamp(self, timestamp: str) -> str:
        try:
            dt = datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
            return dt.strftime("%H:%M:%S")
        except:
            return timestamp
    
    def _get_action_icon(self, action: str) -> str:
        action_icons = {
            'tap': '👆',
            'click': '🖱️',
            'swipe': '👆',
            'scroll': '📜',
            'input': '⌨️',
            'keyevent': '⌨️',
            'long_press': '👆',
            'type': '⌨️',
            'press': '👆',
            'release': '👆',
            'drag': '👆',
            'pinch': '👆',
            'rotate': '🔄'
        }
        return action_icons.get(action.lower(), '⚡')
    
    def _simplify_element_name(self, element_id: str) -> str:
        element_mapping = {
            'primary_action': 'Primary Button',
            'interactive_control': 'Interactive Control',
            'navigation_tap': 'Navigation',
            'edge_tap': 'Edge Area',
            'swipe_scroll': 'Scrollable Content',
            'swipe_menu': 'Menu',
            'swipe_next': 'Next Screen',
            'back_button': 'Back Button',
            'button': 'Button',
            'text': 'Text Element',
            'image': 'Image',
            'link': 'Link',
            'colored_button': 'Colored Button',
            'edge_bounded': 'Edge Bounded Element',
            'center_tap': 'Center Tap', # Added from user provided data
            'secondary_action': 'Secondary Action' # Added from user provided data
        }
        
        return element_mapping.get(element_id, element_id.replace('_', ' ').title())
    
    def _create_descriptive_reasoning(self, event: LLMEvent) -> str:
        if not event or not event.action or not event.element_id:
            return "No action or element information available"
            
        action = event.action.lower()
        element = event.element_id.lower()
        

        # Create contextual descriptions based on element types

In [ ]:
        element_descriptions = {
            'primary_action': 'Main action button or primary interface element',
            'interactive_control': 'Interactive UI control or button',
            'navigation_tap': 'Navigation element or menu item',
            'edge_tap': 'Edge area or boundary element',
            'swipe_scroll': 'Scrollable content area',
            'swipe_menu': 'Menu or drawer that can be swiped',
            'swipe_next': 'Next screen or pagination element',
            'back_button': 'Back navigation or return element',
            'button': 'General button element',
            'text': 'Text input or display element',
            'image': 'Image or visual element',
            'link': 'Clickable link or hyperlink',
            'colored_button': 'a colorful call-to-action button',
            'edge_bounded': 'an element bounded by screen edges',
            'center_tap': 'a central interactive element',
            'secondary_action': 'a secondary action element'
        }
        

        # Create action descriptions

In [ ]:
        action_descriptions = {
            'tap': 'User tapped on',
            'click': 'User clicked on',
            'swipe': 'User swiped',
            'scroll': 'User scrolled',
            'input': 'User entered text in',
            'keyevent': 'User pressed key on',
            'long_press': 'User long-pressed on'
        }
        

        # Get descriptions

In [ ]:
        element_desc = element_descriptions.get(element, f'{element.replace("_", " ")} element')
        action_desc = action_descriptions.get(action, f'User performed {action} on')
        

        # Create contextual reasoning

Analyze features to categorize them as unique or generic.

In [ ]:
        if 'primary' in element:
            return f"{action_desc} the primary action button to proceed with the main workflow"
        elif 'navigation' in element:
            return f"{action_desc} a navigation element to move to a different section"
        elif 'swipe' in element:
            if 'scroll' in element:
                return f"{action_desc} scrollable content to explore more options"
            elif 'menu' in element:
                return f"{action_desc} a menu to reveal additional navigation options"
            elif 'next' in element:
                return f"{action_desc} to advance to the next screen or page"
            elif 'prev' in element:
                return f"{action_desc} to go to the previous screen or page"
        elif 'back' in element:
            return f"{action_desc} the back button to return to the previous screen"
        elif 'interactive' in element:
            return f"{action_desc} an interactive control to engage with the interface"
        elif 'edge' in element:
            return f"{action_desc} an edge area to interact with boundary elements"
        elif 'center_tap' in element:
            return f"{action_desc} center tap element to interact with the interface"
        elif 'secondary_action' in element:
            return f"{action_desc} secondary action element to interact with the interface"
        else:
            return f"{action_desc} {element_desc} to interact with the interface"
    
    def analyze_features(self) -> Tuple[List[str], List[str]]:

        # Collect all text elements from states and events

In [ ]:
        all_text_elements = []
        

        # Collect from LLM events

In [ ]:
        for event in self.llm_events:
            if event.reasoning:
                all_text_elements.append(event.reasoning)
            if event.visual_description:
                all_text_elements.append(event.visual_description)
        

        # Collect from app states

In [ ]:
        for state in self.app_states.values():
            if state.description:
                all_text_elements.append(state.description)
            
            for element in state.elements:
                element_text = element.get('text', '')
                if element_text:
                    all_text_elements.append(element_text)
        

        # Analyze the collected text to identify features

In [ ]:
        unique_features = []
        generic_features = []
        

        # Keywords for categorization - can be expanded

In [ ]:
        unique_keywords = {
            'app-specific': ['custom functionality', 'brand-specific elements', 'specialized services'],

            # Example keywords from previous data (dainikbhaskar implies news/media)

In [ ]:
            'news': ['news', 'article', 'headline', 'daily', 'bhaskar', 'media', 'edition'],
            'content': ['content feed', 'categories', 'latest updates', 'reading mode'],
            'personalization': ['my feed', 'preferences', 'bookmarks']
        }
        
        generic_keywords = [
            'navigation controls', 'login', 'sign in', 'register', 'account', 'profile', 'settings', 'config',
            'search', 'filter', 'sort', 'browse', 'view', 'details',
            'contact', 'help', 'support', 'about', 'privacy', 'terms', 'legal',
            'home', 'back', 'next', 'previous', 'save', 'cancel', 'delete',
            'edit', 'add', 'remove', 'update', 'refresh', 'reload', 'sync'
        ]
        

        # Analyze text elements for unique features

In [ ]:
        app_name_lower = self.app_name.lower()
        package_lower = self.package_name.lower()
        

        # Pre-fill with the unique and generic features found in the original report

In [ ]:
        unique_features_preset = [
            "App-specific features",
            "Custom functionality",
            "Brand-specific elements",
            "Specialized services"
        ]
        generic_features_preset = [
            "Navigation controls"
        ]


        # Add presets to the lists, ensuring no duplicates

In [ ]:
        for f in unique_features_preset:
            if f not in unique_features:
                unique_features.append(f)
        for f in generic_features_preset:
            if f not in generic_features:
                generic_features.append(f)


        # Check for app-specific patterns

In [ ]:
        for text in all_text_elements:
            text_lower = text.lower()
            
            for category, keywords in unique_keywords.items():
                if any(keyword in text_lower for keyword in keywords):

                    # Check if this matches the app's domain (e.g., if "dainikbhaskar" suggests news)

In [ ]:
                    if self._is_app_domain_match(category, app_name_lower, package_lower):
                        feature_name = self._extract_feature_name(text, category)
                        if feature_name and feature_name not in unique_features:
                            unique_features.append(feature_name)
            

            # Check for generic features

In [ ]:
            for keyword in generic_keywords:
                if keyword in text_lower:
                    feature_name = self._extract_generic_feature_name(text, keyword)
                    if feature_name and feature_name not in generic_features:
                        generic_features.append(feature_name)
        

        # If still very few unique features, generate based on common app domains

In [ ]:
        if len(unique_features) < 4:
            generated_features = self._generate_app_specific_features(app_name_lower, package_lower)
            for f in generated_features:
                if f not in unique_features:
                    unique_features.append(f)


        # Ensure we have a reasonable set of generic features

In [ ]:
        if len(generic_features) < 3:
            default_generic = [
                "User authentication",
                "Settings page",
                "Search functionality",
                "Contact information"
            ]
            for f in default_generic:
                if f not in generic_features:
                    generic_features.append(f)
        

        # Limit to reasonable number of features (e.g., top 5-8)

Check if the category matches the app's domain.

Extract a meaningful feature name from text, trying to be more specific.

In [ ]:
        unique_features = list(dict.fromkeys(unique_features))[:8] # Remove duplicates and limit
        generic_features = list(dict.fromkeys(generic_features))[:8] # Remove duplicates and limit
        
        logger.info(f"Found {len(unique_features)} unique features and {len(generic_features)} generic features")
        
        return unique_features, generic_features
    
    def _is_app_domain_match(self, category: str, app_name: str, package_name: str) -> bool:
        category_keywords = {
            'news': ['news', 'media', 'bhaskar', 'live', 'daily', 'paper', 'magazine'],
            'content': ['feed', 'reader', 'story', 'article'],
            'personalization': ['my', 'preferences', 'settings'],
            'app-specific': [app_name.replace(" ", "").lower(), package_name.replace("com.", "").replace(".", "").lower()]
        }
        
        keywords = category_keywords.get(category, [])
        return any(keyword in app_name or keyword in package_name for keyword in keywords)
    
    def _extract_feature_name(self, text: str, category: str) -> str:
        if category == 'news' and 'news' in text.lower():
            return "News content display"
        if category == 'content' and 'feed' in text.lower():
            return "Content feed display"
        if category == 'personalization' and 'profile' in text.lower():
            return "User profile management"
        

        # Fallback to general category description

Extract a generic feature name from text.

Generate app-specific features based on app name and package,

In [ ]:
        return f"{category.replace('-', ' ').title()} functionality"

    def _extract_generic_feature_name(self, text: str, keyword: str) -> str:
        feature_mapping = {
            'login': 'User authentication',
            'sign': 'User authentication',
            'register': 'User registration',
            'account': 'Account management',
            'profile': 'User profile',
            'settings': 'Settings page',
            'config': 'Configuration',
            'search': 'Search functionality',
            'find': 'Search functionality',
            'filter': 'Filter options',
            'sort': 'Sort functionality',
            'browse': 'Browse interface',
            'view': 'View options',
            'details': 'Detail views',
            'contact': 'Contact information',
            'help': 'Help system',
            'support': 'Support features',
            'about': 'About page',
            'privacy': 'Privacy settings',
            'terms': 'Terms of service',
            'legal': 'Legal information',
            'home': 'Home navigation',
            'back': 'Navigation controls',
            'next': 'Navigation controls',
            'previous': 'Navigation controls',
            'save': 'Save functionality',
            'cancel': 'Cancel options',
            'delete': 'Delete functionality',
            'edit': 'Edit functionality',
            'add': 'Add functionality',
            'remove': 'Remove functionality',
            'update': 'Update functionality',
            'refresh': 'Refresh functionality',
            'reload': 'Reload functionality',
            'sync': 'Sync functionality',
            'navigation controls': 'Navigation controls' # From user's report
        }
        
        return feature_mapping.get(keyword, f"{keyword.title()} feature")
    
    def _generate_app_specific_features(self, app_name: str, package_name: str) -> List[str]:
        features = []
        

        # Prioritize news/media features if "dainikbhaskar" or similar is found

In [ ]:
        if any(word in app_name for word in ['bhaskar', 'dainik', 'news', 'daily']) or \
           any(word in package_name for word in ['bhaskar', 'news', 'media']):
            features.extend([
                "Daily News Updates",
                "Article Browse & Reading",
                "Regional News Editions",
                "Live News Feeds",
                "Content Sharing Options"
            ])
        

        # General app features if specific domain not strongly identified

Extract a readable app name from package name.

In [ ]:
        if not features: # Only add these if no specific domain features were added
            features.extend([
                "App-specific features",
                "Custom functionality",
                "Brand-specific elements",
                "Specialized services"
            ])
        
        return features
    
    def _extract_app_name_from_package(self, package_name: str) -> str:
        if not package_name:
            return "Unknown App"
        

        # Common package name patterns

In [ ]:
        package_mapping = {
            'com.dominos': 'Domino\'s Pizza',
            'com.pizzahut': 'Pizza Hut',
            'com.dainikbhaskar': 'Dainik Bhaskar',
            'com.spotify': 'Spotify',
            'com.netflix': 'Netflix',
            'com.uber': 'Uber',
            'com.lyft': 'Lyft',
            'com.amazon': 'Amazon',
            'com.flipkart': 'Flipkart',
            'com.swiggy': 'Swiggy',
            'com.zomato': 'Zomato',
            'com.olivegarden': 'Olive Garden',
            'com.expedia': 'Expedia',
            'com.booking': 'Booking.com',
            'com.airbnb': 'Airbnb'
        }
        

        # Check for exact matches first

In [ ]:
        for package_pattern, app_name in package_mapping.items():
            if package_pattern in package_name.lower():
                return app_name
        

        # Extract from package name structure (com.company.app)

In [ ]:
        parts = package_name.split('.')
        if len(parts) >= 3:

            # Try to get the last meaningful part

In [ ]:
            app_part = parts[-1]
            if app_part not in ['activity', 'main', 'app', 'ui']:
                return app_part.replace('_', ' ').title()
            elif len(parts) >= 2:
                company_part = parts[-2]
                return company_part.replace('_', ' ').title()
        

        # Fallback: clean up the package name

Clean text to be safe for AI processing and encoding.

In [ ]:
        clean_name = package_name.replace('com.', '').replace('_', ' ').title()
        return clean_name
    
    def _clean_text_for_ai(self, text: str) -> str:
        if not text:
            return ""
        
        try:
            import unicodedata
            text = unicodedata.normalize('NFKD', text)
            

            # Replace problematic characters

In [ ]:
            replacements = {
                '“': '"',  # Smart quotes to regular quotes
                '”': '"',
                '‘': "'",  # Smart apostrophes to regular apostrophes
                '’': "'",
                '–': '-',  # En dash to hyphen
                '—': '-',  # Em dash to hyphen
                '…': '...',  # Ellipsis to three dots
                '\u201c': '"',  # Unicode smart quotes
                '\u201d': '"',
                '\u2018': "'",
                '\u2019': "'",
                '\u2013': '-',
                '\u2014': '-',
                '\u2026': '...'
            }
            
            for old, new in replacements.items():
                text = text.replace(old, new)
            

            # Remove any remaining non-ASCII characters that can't be encoded

In [ ]:
            text = text.encode('ascii', 'ignore').decode('ascii')
            

            # Clean up extra whitespace

Generate the complete comprehensive PDF report.
        
        Returns
        -------
        str
            Path to the generated PDF report

In [ ]:
            text = ' '.join(text.split())
            
            return text.strip()
            
        except Exception as e:
            logger.warning(f"Text cleaning failed: {e}")
            return text.encode('ascii', 'ignore').decode('ascii').strip()
    
    def generate_comprehensive_report(self) -> str:
        if not REPORTLAB_AVAILABLE:
            logger.error("ReportLab not available. Cannot generate PDF report.")
            return ""
            
        logger.info("Generating comprehensive final PDF report...")
        

        # Create PDF document

In [ ]:
        report_path = self.output_dir / "final_comprehensive_report.pdf"
        doc = SimpleDocTemplate(
            str(report_path),
            pagesize=A4,
            rightMargin=72,
            leftMargin=72,
            topMargin=72,
            bottomMargin=72
        )
        

        # Define custom styles

In [ ]:
        styles = getSampleStyleSheet()
        

        # Title style

In [ ]:
        title_style = ParagraphStyle(
            'CustomTitle',
            parent=styles['Heading1'],
            fontSize=28,
            spaceAfter=30,
            alignment=TA_CENTER,
            textColor=colors.HexColor('#0A2F5B'), # Dark Blue
            leading=30
        )
        

        # Main heading style for sections (e.g., Executive Summary)

In [ ]:
        heading1_style = ParagraphStyle(
            'CustomHeading1',
            parent=styles['Heading1'],
            fontSize=18,
            spaceAfter=12,
            spaceBefore=20,
            textColor=colors.HexColor('#0A2F5B'), # Dark Blue
            alignment=TA_LEFT,
            fontName='Helvetica-Bold'
        )
        

        # Sub-heading style (e.g., Test Statistics)

In [ ]:
        heading2_style = ParagraphStyle(
            'CustomHeading2',
            parent=styles['Heading2'],
            fontSize=14,
            spaceAfter=8,
            spaceBefore=16,
            textColor=colors.HexColor('#2E5B8E'), # Medium Blue
            alignment=TA_LEFT,
            fontName='Helvetica-Bold'
        )


        # Smaller heading style (e.g., Event #1)

In [ ]:
        heading3_style = ParagraphStyle(
            'CustomHeading3',
            parent=styles['Heading3'],
            fontSize=12,
            spaceAfter=6,
            spaceBefore=10,
            textColor=colors.HexColor('#4A7BA8'), # Lighter Blue
            alignment=TA_LEFT,
            fontName='Helvetica-Bold'
        )
        

        # Normal text style

In [ ]:
        normal_style = ParagraphStyle(
            'CustomNormal',
            parent=styles['Normal'],
            fontSize=10,
            spaceAfter=6,
            alignment=TA_JUSTIFY,
            leading=12
        )


        # Bold normal text for emphasis

In [ ]:
        bold_normal_style = ParagraphStyle(
            'BoldNormal',
            parent=normal_style,
            fontName='Helvetica-Bold'
        )


        # List item style

In [ ]:
        list_style = ParagraphStyle(
            'ListItem',
            parent=normal_style,
            leftIndent=20,
            firstLineIndent=-10,
            bulletIndent=10,
            bulletFontName='Helvetica-Bold',
            bulletFontSize=10,
            bulletColor=colors.HexColor('#4A7BA8'),
            alignment=TA_LEFT
        )
        

        # Build the story (content)

In [ ]:
        story = []
        

        # Title page

In [ ]:
        story.append(Spacer(1, 2 * inch)) # Adjust spacing for centering
        story.append(Paragraph(f"{self.app_name} Test Report", title_style))
        story.append(Spacer(1, 0.5 * inch))
        story.append(Paragraph(f"<b>App:</b> {self.app_name}", bold_normal_style))
        story.append(Paragraph(f"<b>Package:</b> {self.package_name}", bold_normal_style))
        story.append(Paragraph(f"<b>Generated:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", bold_normal_style))
        story.append(Paragraph(f"<b>Test Duration:</b> {self.get_test_duration()}", bold_normal_style))
        story.append(PageBreak())


        # Executive Summary

In [ ]:
        story.extend(self._generate_executive_summary(heading1_style, normal_style))
        story.append(PageBreak()) # Start new page after executive summary


        # Test Statistics

In [ ]:
        story.extend(self._generate_test_statistics(heading1_style, normal_style, bold_normal_style))
        story.append(PageBreak()) # Start new page


        # Feature Analysis

In [ ]:
        story.extend(self._generate_feature_analysis_pdf(heading1_style, heading2_style, list_style))
        story.append(PageBreak()) # Start new page


        # LLM Event Log

In [ ]:
        story.extend(self._generate_llm_event_log_pdf(heading1_style, heading3_style, normal_style))

        # No PageBreak here, as events might span multiple pages

        # OCR Screenshots Section

In [ ]:
        story.append(PageBreak()) # Start new page
        story.extend(self._generate_ocr_screenshots_pdf(heading1_style, heading3_style, normal_style))


        # Analysis & Recommendations

In [ ]:
        story.append(PageBreak()) # Start new page
        story.extend(self._generate_analysis_recommendations_pdf(heading1_style, heading2_style, normal_style, bold_normal_style))
        

        # Helper to add header & footer on each page

In [ ]:
        def _add_header_footer(canvas_obj, doc_obj):
            canvas_obj.saveState()

            # Header

In [ ]:
            canvas_obj.setFont("Helvetica-Bold", 10)
            canvas_obj.setFillColor(colors.HexColor('#0A2F5B'))
            canvas_obj.drawString(doc_obj.leftMargin, A4[1] - 40, f"{self.app_name} – Automated Test Report")

            # Footer

In [ ]:
            canvas_obj.setFont("Helvetica", 9)
            canvas_obj.setFillColor(colors.grey)
            page_num_text = f"Page {doc_obj.page}"
            canvas_obj.drawRightString(A4[0] - doc_obj.rightMargin, 30, page_num_text)
            canvas_obj.restoreState()


        # Build the PDF with header/footer callbacks

Generate executive summary section for PDF.

In [ ]:
        doc.build(story, onFirstPage=_add_header_footer, onLaterPages=_add_header_footer)
        
        logger.info(f"✓ Final comprehensive PDF report generated: {report_path}")
        return str(report_path)
    
    def _generate_executive_summary(self, heading1_style, normal_style):
        story = []
        
        story.append(Paragraph("Executive Summary", heading1_style))
        story.append(Spacer(1, 12))
        

        # Calculate key metrics


        This report presents the results of automated testing for <b>{self.app_name}</b> ({self.package_name}). 
        The AA_VA system successfully explored <b>{total_states}</b> unique app states through <b>{total_actions}</b> 
        automated actions across <b>{total_activities}</b> activities, interacting with <b>{unique_elements}</b> 
        distinct UI elements over a period of <b>{test_duration}</b>.
        
        The testing session utilized advanced computer vision and AI-driven decision making to 
        systematically explore the application's interface, identifying both app-specific features 
        and common UI patterns. The system demonstrated effective navigation through complex 
        user interfaces while maintaining detailed logs of all interactions and state transitions.

Generate test statistics section for PDF.

In [ ]:
        total_states = len(self.app_states)
        total_actions = self.total_actions
        total_activities = len(self.activities)
        unique_elements = len(self.unique_elements_interacted)
        test_duration = self.get_test_duration()
        
        
        story.append(Paragraph(summary_text, normal_style))
        story.append(Spacer(1, 18)) # More space after summary
        
        return story
    
    def _generate_test_statistics(self, heading1_style, normal_style, bold_normal_style):
        story = []
        
        story.append(Paragraph("Test Statistics", heading1_style))
        story.append(Spacer(1, 12))
        

        # Create statistics table

Generate LLM event log section for PDF with improved formatting.

In [ ]:
        stats_data = [
            [Paragraph("<b>Metric</b>", bold_normal_style), Paragraph("<b>Value</b>", bold_normal_style)], # Bold headers
            ["Total States Explored", str(len(self.app_states))],
            ["Total Actions Performed", str(self.total_actions)],
            ["Total Activities Discovered", str(len(self.activities))],
            ["Unique Elements Interacted", str(len(self.unique_elements_interacted))],
            ["Total LLM Events", str(len(self.llm_events))],
            ["Test Duration", self.get_test_duration()]
        ]
        
        stats_table = Table(stats_data, colWidths=[3*inch, 2*inch])
        stats_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#0A2F5B')), # Dark blue header
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke), # White text for header
            ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 11),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 8), # Reduced padding
            ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#F0F8FF')), # Light blue background for rows
            ('GRID', (0, 0), (-1, -1), 0.5, colors.HexColor('#CCCCCC')), # Lighter grid lines
            ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 1), (-1, -1), 10),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('LEFTPADDING', (0,0), (-1,-1), 8),
            ('RIGHTPADDING', (0,0), (-1,-1), 8),
            ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.HexColor('#FFFFFF'), colors.HexColor('#F0F8FF')]) # Alternating row colors
        ]))
        
        story.append(stats_table)
        story.append(Spacer(1, 18))
        
        return story
    
    def _generate_llm_event_log_pdf(self, heading1_style, heading3_style, normal_style):
        story = []
        

        # Keep section header with initial content

In [ ]:
        section_header = [
            Paragraph("🤖 LLM Event Log", heading1_style),
            Spacer(1, 12)
        ]
        
        if not self.llm_events:
            section_header.append(Paragraph("No LLM events recorded during testing.", normal_style))
            story.extend(section_header)
            return story
        

        # Add summary statistics

In [ ]:
        total_events = len(self.llm_events)
        unique_actions = len(set(event.action for event in self.llm_events))
        unique_elements = len(set(event.element_id for event in self.llm_events))
        
        summary_text = f"<b>Summary:</b> {total_events} total events, {unique_actions} unique actions, {unique_elements} unique elements"
        section_header.append(Paragraph(summary_text, normal_style))
        section_header.append(Spacer(1, 12))
        

        # Keep section header together and add to story

In [ ]:
        story.extend(section_header)
        

        # Add summary statistics

In [ ]:
        total_events = len(self.llm_events)
        unique_actions = len(set(event.action for event in self.llm_events))
        unique_elements = len(set(event.element_id for event in self.llm_events))
        
        summary_text = f"<b>Summary:</b> {total_events} total events, {unique_actions} unique actions, {unique_elements} unique elements"
        story.append(Paragraph(summary_text, normal_style))
        story.append(Spacer(1, 12))
        

        # Process events in batches to avoid overwhelming the PDF

In [ ]:
        for i, event in enumerate(self.llm_events, 1):

            # Create event content that should stay together

In [ ]:
            event_content = []
            

            # Create a more visually appealing event header

In [ ]:
            event_header = f"📱 Event #{i:02d} • {self._format_timestamp(event.timestamp)}"
            event_content.append(Paragraph(event_header, heading3_style))
            event_content.append(Spacer(1, 6))
            

            # Create event details with improved layout

In [ ]:
            reasoning = self._create_descriptive_reasoning(event)
            if reasoning is None:
                reasoning = "No reasoning available"
            elif len(reasoning) > 150: # Increased limit for better readability
                reasoning = reasoning[:147] + "..."
                

            # Enhanced event details with better categorization and action-specific icons

In [ ]:
            action_icon = self._get_action_icon(event.action)
            event_details = [
                ["🕒 Timestamp", self._format_timestamp(event.timestamp)],
                [f"{action_icon} Action", f"<b>{event.action.upper()}</b>"],
                ["🎯 Element", f"<b>{self._simplify_element_name(event.element_id)}</b>"],
                ["🔄 State Flow", f"<b>{event.state_id}</b> → <b>{event.resulting_state_id}</b>"],
                ["💭 Reasoning", reasoning]
            ]
            

            # Add input text if available

In [ ]:
            if event.input_text and event.input_text.strip():
                event_details.append(["📝 Input", f"<b>{event.input_text}</b>"])
            

            # Add scroll direction if available

In [ ]:
            if event.scroll_direction and event.scroll_direction.strip():
                event_details.append(["📜 Scroll", f"<b>{event.scroll_direction}</b>"])
            

            # Create table with improved styling

In [ ]:
            event_table = Table(event_details, colWidths=[1.8*inch, 4.2*inch])
            event_table.setStyle(TableStyle([

                # Header styling

In [ ]:
                ('BACKGROUND', (0, 0), (0, -1), colors.HexColor('#F8F9FA')),
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#E9ECEF')),
                

                # Text styling

In [ ]:
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                ('FONTNAME', (0, 0), (0, -1), 'Helvetica-Bold'),
                ('FONTNAME', (1, 0), (1, -1), 'Helvetica'),
                ('FONTSIZE', (0, 0), (-1, -1), 10),
                

                # Grid styling

In [ ]:
                ('GRID', (0, 0), (-1, -1), 0.5, colors.HexColor('#DEE2E6')),
                ('ROWBACKGROUNDS', (0, 0), (-1, -1), [colors.HexColor('#FFFFFF'), colors.HexColor('#F8F9FA')]),
                

                # Spacing

In [ ]:
                ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                ('LEFTPADDING', (0, 0), (-1, -1), 8),
                ('RIGHTPADDING', (0, 0), (-1, -1), 8),
                ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
                ('TOPPADDING', (0, 0), (-1, -1), 6),
                

                # Border styling

In [ ]:
                ('BOX', (0, 0), (-1, -1), 1, colors.HexColor('#6C757D')),
                ('LINEBELOW', (0, 0), (-1, 0), 1, colors.HexColor('#495057')),
            ]))
            
            event_content.append(event_table)
            event_content.append(Spacer(1, 16)) # Increased spacing between events
            

            # Add screenshot if available (only for first few events for brevity)

In [ ]:
            if i <= 2 and event.screenshot_path and os.path.exists(event.screenshot_path):
                try:

                    # Add screenshot with caption

In [ ]:
                    event_content.append(Paragraph("<b>📸 Screenshot:</b>", normal_style))
                    event_content.append(Spacer(1, 3))
                    

                    # Using ImageReader with better sizing

In [ ]:
                    img = Image(event.screenshot_path, width=3.5*inch, height=2.6*inch, kind='bound')
                    event_content.append(img)
                    event_content.append(Spacer(1, 8))
                except Exception as e:
                    logger.warning(f"Could not add screenshot {event.screenshot_path}: {e}")
            

            # Add visual separator between events (except for the last one)

In [ ]:
            if i < len(self.llm_events):
                from reportlab.platypus import HRFlowable
                event_content.append(HRFlowable(width="100%", color=colors.HexColor('#E9ECEF'), thickness=1))
                event_content.append(Spacer(1, 8))
            

            # Keep each event together and add to story

In [ ]:
            story.append(KeepTogether(event_content))
            

            # Add a PageBreak after every 6 events (reduced frequency)

Generate feature analysis section for PDF.

In [ ]:
            if i % 6 == 0 and i < len(self.llm_events):
                story.append(PageBreak())
                story.append(KeepTogether([
                    Paragraph("🤖 LLM Event Log (Continued)", heading1_style),
                    Spacer(1, 12)
                ]))
        
        return story
    
    def _generate_feature_analysis_pdf(self, heading1_style, heading2_style, list_style):
        story = []
        
        story.append(Paragraph("Feature Analysis", heading1_style))
        story.append(Spacer(1, 12))
        
        unique_features, generic_features = self.analyze_features()
        

        # Unique features

In [ ]:
        story.append(Paragraph("Unique Features (App-Specific)", heading2_style))
        story.append(Spacer(1, 6))
        
        for feature in unique_features:
            story.append(Paragraph(f"• {feature}", list_style)) # Using list_style for bullet points
        
        story.append(Spacer(1, 12))
        

        # Generic features

Generate a high-level analysis & recommendations section using all detected UI data.

In [ ]:
        story.append(Paragraph("Generic Features (Common Patterns)", heading2_style))
        story.append(Spacer(1, 6))
        
        for feature in generic_features:
            story.append(Paragraph(f"• {feature}", list_style)) # Using list_style for bullet points
        
        return story

    def _generate_analysis_recommendations_pdf(self, heading1_style, heading2_style, normal_style, bold_normal_style):
        story = []
        story.append(Paragraph("Analysis & Recommendations", heading1_style))
        story.append(Spacer(1, 12))


        # Aggregate metrics

In [ ]:
        total_elements = 0
        type_counts: Dict[str, int] = {}
        confidences = []
        skip_button_detected = False

        for state in self.app_states.values():
            for el in state.elements:
                total_elements += 1
                el_type = el.get("element_type", "unknown")
                type_counts[el_type] = type_counts.get(el_type, 0) + 1
                if "skip" in (el.get("text") or "").lower():
                    skip_button_detected = True
                conf = el.get("confidence")
                if conf is not None:
                    confidences.append(conf)

        avg_confidence = sum(confidences) / len(confidences) if confidences else 0.0


        # Top 3 most common element types

In [ ]:
        sorted_types = sorted(type_counts.items(), key=lambda x: x[1], reverse=True)
        top_elements_summary = ", ".join([f"'{t.replace('_', ' ').title()}' ({c})" for t, c in sorted_types[:3]]) if sorted_types else "N/A"


        # 1. UI/UX analysis (app-focused wording)

In [ ]:
        story.append(Paragraph("1. UI/UX analysis and observations:", heading2_style))
        from reportlab.platypus import HRFlowable
        story.append(HRFlowable(width="100%", color=colors.HexColor('#0A2F5B')))
        ui_ux_text = (
            f"Across the journey we captured roughly <b>{total_elements}</b> distinct on-screen components. "
            f"Most frequently the interface presents {top_elements_summary}, signalling a design that leans on colourful "
            "calls-to-action and text-driven content. While this richness gives users plenty to engage with, the sheer density "
            "may compete for attention and slow decision-making, particularly on smaller screens."
        )
        story.append(Paragraph(ui_ux_text, normal_style))
        story.append(Spacer(1, 6))


        # 2. Accessibility improvements

In [ ]:
        story.append(Paragraph("2. Potential accessibility improvements:", heading2_style))
        story.append(HRFlowable(width="100%", color=colors.HexColor('#0A2F5B')))
        accessibility_text = (
            "Vibrant buttons instantly draw the eye, but ensure contrast ratios meet WCAG 2.2 so that colour-blind visitors "
            "can still distinguish primary actions. Typography should remain comfortably legible at typical viewing distances. "
            + ("Adding a clear <b>“Skip”</b> or <b>“Next”</b> option would further empower users who prefer to fast-track onboarding or promotions." if not skip_button_detected else "A handy <b>“Skip”</b> button already appears in certain flows, giving users quick control.")
        )
        story.append(Paragraph(accessibility_text, normal_style))
        story.append(Spacer(1, 6))


        # 3. Automation opportunities

In [ ]:
        story.append(Paragraph("3. Automation opportunities:", heading2_style))
        story.append(HRFlowable(width="100%", color=colors.HexColor('#0A2F5B')))
        auto_text = (
            "Consistency across screens makes the experience predictable for both users and any automated quality checks. "
            "Interactive buttons, edge-anchored drawers and recurring text inputs lend themselves to scripted walkthroughs "
            "that safeguard core journeys release after release."
        )
        story.append(Paragraph(auto_text, normal_style))
        story.append(Spacer(1, 6))


        # 4. Testing recommendations

In [ ]:
        story.append(Paragraph("4. Testing recommendations:", heading2_style))
        story.append(HRFlowable(width="100%", color=colors.HexColor('#0A2F5B')))
        test_rec_text = (
            "Invite real users to timed tasks to learn whether they notice cognitive overload or friction navigating between key screens. "
            "Supplement sessions with quick-hit accessibility scans (WCAG 2.2): colour contrast, touch-target sizing and dynamic-type support. "
            "Finally, profile rendering and scrolling to confirm the interface stays fluid even when many elements populate the view hierarchy."
        )
        story.append(Paragraph(test_rec_text, normal_style))
        story.append(Spacer(1, 6))


        # 5. Overall quality assessment

Generate OCR screenshots section for PDF.

In [ ]:
        story.append(Paragraph("5. Overall app quality assessment:", heading2_style))
        story.append(HRFlowable(width="100%", color=colors.HexColor('#0A2F5B')))
        qual_text = (
            "Overall the app feels lively and feature-packed. Refining hierarchy, trimming redundant elements and strengthening "
            "navigational cues will elevate first impressions and day-to-day usability for a broad audience." 
        )
        story.append(Paragraph(qual_text, normal_style))
        story.append(Spacer(1, 18))

        return story

    def _generate_ocr_screenshots_pdf(self, heading1_style, heading3_style, normal_style):
        story = []
        

        # Keep section header with initial content

In [ ]:
        section_header = [
            Paragraph("🔍 OCR Screenshots", heading1_style),
            Spacer(1, 12)
        ]
        

        # Check if OCR images directory exists

In [ ]:
        ocr_images_dir = self.output_dir / "ocr_images"
        if not ocr_images_dir.exists():
            section_header.append(Paragraph("No OCR screenshots available for this test session.", normal_style))
            section_header.append(Spacer(1, 12))
            section_header.append(Paragraph("OCR screenshots show text detection with bounding boxes around detected UI elements.", normal_style))
            story.extend(section_header)
            return story
        

        # Find all OCR image files

In [ ]:
        ocr_files = list(ocr_images_dir.glob("*.png"))
        if not ocr_files:
            section_header.append(Paragraph("No OCR screenshots found in the ocr_images directory.", normal_style))
            section_header.append(Spacer(1, 12))
            section_header.append(Paragraph("OCR screenshots show text detection with bounding boxes around detected UI elements.", normal_style))
            story.extend(section_header)
            return story
        

        # Sort files by name to maintain order

In [ ]:
        ocr_files.sort(key=lambda x: x.name)
        

        # Add description

In [ ]:
        description_text = (
            "The following OCR screenshots show text detection results with bounding boxes around detected UI elements. "
            "These images demonstrate how the computer vision system identifies and extracts text from the app interface "
            "during automated testing."
        )
        section_header.append(Paragraph(description_text, normal_style))
        section_header.append(Spacer(1, 12))
        

        # Add summary statistics

In [ ]:
        total_ocr_images = len(ocr_files)
        section_header.append(Paragraph(f"<b>Total OCR Screenshots:</b> {total_ocr_images}", normal_style))
        section_header.append(Spacer(1, 12))
        

        # Keep section header together and add to story

In [ ]:
        story.extend(section_header)
        

        # Limit to 25 screenshots and process in rows of 3

In [ ]:
        max_screenshots = 25
        images_per_row = 3
        ocr_files = ocr_files[:max_screenshots]  # Limit to first 25
        

        # Add note if more screenshots exist

In [ ]:
        if len(ocr_files) == max_screenshots and total_ocr_images > max_screenshots:
            note_text = f"<i>Note: Showing first {max_screenshots} OCR screenshots out of {total_ocr_images} total.</i>"
            story.append(Paragraph(note_text, normal_style))
            story.append(Spacer(1, 12))
        

        # Process OCR images in rows of 3

In [ ]:
        for i in range(0, len(ocr_files), images_per_row):
            row_files = ocr_files[i:i + images_per_row]
            

            # Create row content that should stay together

In [ ]:
            row_content = []
            

            # Create row header

In [ ]:
            row_start = i + 1
            row_end = min(i + images_per_row, len(ocr_files))
            row_header = f"📸 OCR Screenshots #{row_start:02d}-#{row_end:02d}"
            row_content.append(Paragraph(row_header, heading3_style))
            row_content.append(Spacer(1, 6))
            

            # Create table for this row of images

In [ ]:
            row_data = []
            for j, ocr_file in enumerate(row_files):
                try:

                    # Create image with larger size for 3-column layout

In [ ]:
                    img = Image(str(ocr_file), width=2.6*inch, height=1.95*inch, kind='bound')
                    

                    # Create cell content with image and file info

In [ ]:
                    cell_content = [
                        img,
                        Paragraph(f"<b>{ocr_file.stem}</b>", normal_style),
                        Paragraph(f"Size: {ocr_file.stat().st_size / 1024:.1f} KB", normal_style)
                    ]
                    row_data.append(cell_content)
                    
                except Exception as e:
                    logger.warning(f"Could not add OCR image {ocr_file}: {e}")
                    cell_content = [
                        Paragraph(f"⚠️ Error loading image", normal_style),
                        Paragraph(f"<b>{ocr_file.name}</b>", normal_style),
                        Paragraph("File not available", normal_style)
                    ]
                    row_data.append(cell_content)
            

            # Pad row with empty cells if needed

In [ ]:
            while len(row_data) < images_per_row:
                row_data.append([
                    Paragraph("", normal_style),
                    Paragraph("", normal_style),
                    Paragraph("", normal_style)
                ])
            

            # Create table for this row

In [ ]:
            row_table = Table(row_data, colWidths=[2.8*inch, 2.8*inch, 2.8*inch])
            row_table.setStyle(TableStyle([
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('VALIGN', (0, 0), (-1, -1), 'TOP'),
                ('LEFTPADDING', (0, 0), (-1, -1), 5),
                ('RIGHTPADDING', (0, 0), (-1, -1), 5),
                ('BOTTOMPADDING', (0, 0), (-1, -1), 5),
                ('TOPPADDING', (0, 0), (-1, -1), 5),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.HexColor('#DEE2E6')),
            ]))
            
            row_content.append(row_table)
            row_content.append(Spacer(1, 12))
            

            # Keep each row together and add to story

In [ ]:
            story.append(KeepTogether(row_content))
            

            # Add page break after every 4 rows (12 images) for better spacing

Main function to demonstrate the final report generator.

In [ ]:
            if (i + images_per_row) % (images_per_row * 4) == 0 and (i + images_per_row) < len(ocr_files):
                story.append(PageBreak())
                story.append(KeepTogether([
                    Paragraph("🔍 OCR Screenshots (Continued)", heading1_style),
                    Spacer(1, 12)
                ]))

        return story


def main():
    import argparse
    
    parser = argparse.ArgumentParser(description="Generate Final Comprehensive PDF Report")
    parser.add_argument("--output-dir", default=".",
                       help="Base output directory for the final report (will create test_com_appname subdirectory)")
    parser.add_argument("--app-name", default=None,
                       help="App name (will auto-extract from package if not provided)")
    parser.add_argument("--package-name", default="com.ak.ta.dainikbhaskar.activity",
                       help="Package name of the app")
    parser.add_argument("--demo", action="store_true",
                       help="Run with demo data")
    
    args = parser.parse_args()
    

    # Initialize generator with dynamic app name extraction

In [ ]:
    generator = FinalReportGenerator(
        output_dir=args.output_dir,
        app_name=args.app_name,
        package_name=args.package_name
    )
    

    # Set the test duration based on the provided report

In [ ]:
    test_start = datetime(2025, 7, 19, 17, 54, 11) # Adjusted for the 37s duration ending at 17:54:48
    test_end = datetime(2025, 7, 19, 17, 54, 48)
    generator.set_test_start_time(test_start)
    generator.set_test_end_time(test_end)

    if args.demo:

        # Add demo data matching the provided report structure

In [ ]:
        logger.info("Adding demo data mimicking the provided report...")
        

        # Add demo App States (Total 17)

In [ ]:
        for i in range(17):
            state = AppState(
                state_id=f"state_{i}",
                activity_name=generator.package_name, # Use the actual package name
                description=f"App state {i} description.",
                screenshot_path=os.path.join(generator.output_dir, "images", f"screenshot_state_{i}.png"), # Placeholder
                elements=[] # Can populate with more details if needed for analysis
            )
            generator.add_app_state(state)
        

        # Populate elements for UI/UX analysis based on the report

In [ ]:
        if "state_0" in generator.app_states:
            generator.app_states["state_0"].elements.extend([
                {"element_type": "colored_button", "text": "Proceed", "confidence": 0.95},
                {"element_type": "edge_bounded", "text": "Top Bar", "confidence": 0.88},
                {"element_type": "button", "text": "Menu", "confidence": 0.90}
            ])

        # Add more specific elements if needed for feature analysis accuracy
        # For simplicity, will use general element types as indicated in the report for analysis
        # Adding elements for the analysis to pick up specific keywords

In [ ]:
        if "state_1" in generator.app_states:
            generator.app_states["state_1"].elements.extend([
                {"element_type": "navigation_tap", "text": "Home", "confidence": 0.92},
                {"element_type": "colored_button", "text": "Explore Now", "confidence": 0.96},
                {"element_type": "text", "text": "Daily News Digest", "confidence": 0.85}
            ])
        if "state_2" in generator.app_states:
            generator.app_states["state_2"].elements.extend([
                {"element_type": "center_tap", "text": "View Article", "confidence": 0.91},
                {"element_type": "navigation_tap", "text": "Categories", "confidence": 0.89},
                {"element_type": "button", "text": "Search", "confidence": 0.87}
            ])
        if "state_3" in generator.app_states:
            generator.app_states["state_3"].elements.extend([
                {"element_type": "secondary_action", "text": "Share", "confidence": 0.88},
                {"element_type": "text", "text": "Breaking News", "confidence": 0.90}
            ])

        # Add a "skip" button to some state for testing accessibility recommendation logic

In [ ]:
        if "state_4" in generator.app_states:
            generator.app_states["state_4"].elements.extend([
                {"element_type": "button", "text": "Skip Tutorial", "confidence": 0.90}
            ])


        # Add LLM Events (Total 16)

In [ ]:
        llm_events_data = [
            {"timestamp": "17:54:28", "state_id": "state_0", "element_id": "primary_action", "action": "tap", "resulting_state_id": "state_1", "reasoning": "User tapped on the primary action button to proceed with the main workflow", "screenshot_path": "page3_img1.png"},
            {"timestamp": "17:54:28", "state_id": "state_1", "element_id": "primary_action", "action": "tap", "resulting_state_id": "state_1", "reasoning": "User tapped on the primary action button to proceed with the main workflow", "screenshot_path": "page3_img2.png"},
            {"timestamp": "17:54:30", "state_id": "state_1", "element_id": "navigation_tap", "action": "tap", "resulting_state_id": "state_2", "reasoning": "User tapped on a navigation element to move to a different section", "screenshot_path": "page4_img1.png"},
            {"timestamp": "17:54:30", "state_id": "state_2", "element_id": "navigation_tap", "action": "tap", "resulting_state_id": "state_2", "reasoning": "User tapped on a navigation element to move to a different section", "screenshot_path": "page4_img2.png"},
            {"timestamp": "17:54:32", "state_id": "state_2", "element_id": "center_tap", "action": "tap", "resulting_state_id": "state_3", "reasoning": "User tapped on center tap element to interact with the interface", "screenshot_path": "page4_img3.png"},
            {"timestamp": "17:54:33", "state_id": "state_3", "element_id": "secondary_action", "action": "tap", "resulting_state_id": "state_4", "reasoning": "User tapped on secondary action element to interact with the interface"},
            {"timestamp": "17:54:35", "state_id": "state_4", "element_id": "navigation_tap", "action": "tap", "resulting_state_id": "state_5", "reasoning": "User tapped on a navigation element to move to a different section"},
            {"timestamp": "17:54:35", "state_id": "state_5", "element_id": "navigation_tap", "action": "tap", "resulting_state_id": "state_5", "reasoning": "User tapped on a navigation element to move to a different section"},
            {"timestamp": "17:54:38", "state_id": "state_5", "element_id": "swipe_prev", "action": "swipe", "resulting_state_id": "state_6", "reasoning": "No reasoning available", "screenshot_path": "page5_img1.png"},
            {"timestamp": "17:54:38", "state_id": "state_6", "element_id": "swipe_prev", "action": "swipe", "resulting_state_id": "state_6", "reasoning": "No reasoning available"},
            {"timestamp": "17:54:40", "state_id": "state_6", "element_id": "edge_tap", "action": "tap", "resulting_state_id": "state_7", "reasoning": "User tapped on an edge area to interact with boundary elements"},
            {"timestamp": "17:54:42", "state_id": "state_7", "element_id": "navigation_tap", "action": "tap", "resulting_state_id": "state_8", "reasoning": "User tapped on a navigation element to move to a different section"},
            {"timestamp": "17:54:42", "state_id": "state_8", "element_id": "navigation_tap", "action": "tap", "resulting_state_id": "state_8", "reasoning": "User tapped on a navigation element to move to a different section"},
            {"timestamp": "17:54:44", "state_id": "state_8", "element_id": "swipe_menu", "action": "swipe", "resulting_state_id": "state_9", "reasoning": "User swiped a menu to reveal additional navigation options"},
            {"timestamp": "17:54:47", "state_id": "state_9", "element_id": "primary_action", "action": "tap", "resulting_state_id": "state_10", "reasoning": "User tapped on the primary action button to proceed with the main workflow"},
            {"timestamp": "17:54:47", "state_id": "state_10", "element_id": "primary_action", "action": "tap", "resulting_state_id": "state_10", "reasoning": "User tapped on the primary action button to proceed with the main workflow"}
        ]


        # Create dummy image files for screenshots

In [ ]:
        for event_data in llm_events_data:
            if "screenshot_path" in event_data and event_data["screenshot_path"]:
                dummy_image_path = os.path.join(generator.output_dir, "images", event_data["screenshot_path"])

                # Create a dummy image file (e.g., a blank PNG)

In [ ]:
                from PIL import Image as PILImage
                PILImage.new('RGB', (400, 600), color = 'lightgrey').save(dummy_image_path)
                event_data["screenshot_path"] = dummy_image_path # Update path to absolute

        for event_data in llm_events_data:
            event = LLMEvent(
                timestamp=event_data["timestamp"],
                state_id=event_data["state_id"],
                element_id=event_data["element_id"],
                action=event_data["action"],
                resulting_state_id=event_data["resulting_state_id"],
                reasoning=event_data["reasoning"],
                screenshot_path=event_data.get("screenshot_path", "")
            )
            generator.add_llm_event(event)

    

    # Generate the comprehensive report

In [ ]:
    report_path = generator.generate_comprehensive_report()
    
    logger.info("🎉 Final comprehensive PDF report generated successfully!")
    logger.info(f"📊 Report available at: {report_path}")
    logger.info("📋 Report includes:")
    logger.info("  - Executive Summary")
    logger.info("  - Test Statistics")
    logger.info("  - LLM Event Log")
    logger.info("  - Feature Analysis")
    logger.info("  - Analysis & Recommendations")


if __name__ == "__main__":
    main()